In [3]:
import geopandas as gpd
from shapely.geometry import LineString
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd

# 1. Load trimmed lines
gdf = gpd.read_file("ortodromic_erope_trimmed.gpkg")

# 2. Color interpolation
def interpolate_color(start_color, end_color, t):
    start_rgb = np.array(mcolors.to_rgb(start_color))
    end_rgb = np.array(mcolors.to_rgb(end_color))
    interp_rgb = (1 - t) * start_rgb + t * end_rgb
    return mcolors.to_hex(interp_rgb)

# 3. Segmenting lines
def segment_line(row, n_segments=50):
    line = row.geometry
    pct = row.pct

    if not isinstance(line, LineString) or line.length == 0 or pd.isna(pct):
        return []

    segments = []
    for i in range(n_segments):
        start_frac = i / n_segments
        end_frac = (i + 1) / n_segments
        start_point = line.interpolate(start_frac, normalized=True)
        end_point = line.interpolate(end_frac, normalized=True)
        segment = LineString([start_point, end_point])

        t = i / (n_segments - 1)
        if pct >= 50:
            color = interpolate_color("#e31a1c", "#dad18d", t)  # red → neutral
        else:
            color = interpolate_color("#33a02c", "#dad18d", t)  # green → neutral

        segments.append({
            'origin': row.get('origin'),
            'destination': row.get('destination'),
            'value': row.get('value'),
            'value_total': row.get('value_total'),
            'pct': pct,
            'geometry': segment,
            'color': color
        })
    return segments

# 4. Apply segmentation
all_segments = []
for _, row in gdf.iterrows():
    all_segments.extend(segment_line(row))

# 5. Create GeoDataFrame
segments_gdf = gpd.GeoDataFrame(all_segments, geometry='geometry', crs=gdf.crs)

# 6. Save output
segments_gdf.to_file("ortodromic_erope_gradient_segments.gpkg", driver="GPKG", layer="segments_with_color")

print("✅ Export completed to 'ortodromic_erope_gradient_segments.gpkg'")


✅ Export completed to 'ortodromic_erope_gradient_segments.gpkg'
